In [3]:
import pandas as pd
from pathlib import Path

In [7]:
df = pd.read_csv(Path.home() / r"Box\Fed-Register\final_rules_all_data.csv")
df['authors'] = df['authors'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df['regulation_id_numbers'] = df['regulation_id_numbers'].apply(lambda x: eval(x) if isinstance(x, str) else x)
df['agencies'] = df['agencies'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Replace publisher names in an attempt to normalize them

In [28]:
publisher_counts = df['publisher'].value_counts()

In [29]:
df2 = df.explode('authors')

In [31]:
author_counts = df2['authors'].value_counts()

In [32]:
auth_agencies = df2.explode('agencies')
pub_agencies = df.explode('agencies')

In [33]:
auth_agency = auth_agencies[['authors', 'agencies']].value_counts().reset_index()
pub_agency = pub_agencies[['publisher', 'agencies']].value_counts().reset_index()

In [ ]:
# Keep top 5 authors for each agency
top_authors_per_agency = auth_agency.groupby('agencies').apply(lambda x: x.nlargest(5, 'count')).reset_index(drop=True)

# Keep top 5 publishers for each agency
top_publishers_per_agency = pub_agency.groupby('agencies').apply(lambda x: x.nlargest(5, 'count')).reset_index(drop=True)

In [34]:
titles = df['title'].value_counts()

In [37]:
dir = Path(r"C:\Users\svens\Box\Fed-Register\Tables")
with pd.ExcelWriter(dir / 'output.xlsx') as writer:
    titles.iloc[:10].to_excel(writer, sheet_name='Titles')
    author_counts.iloc[:10].to_excel(writer, sheet_name='Authors')
    publisher_counts.iloc[:10].to_excel(writer, sheet_name='Publishers')
    top_authors_per_agency.to_excel(writer, sheet_name='Authors_by_Agency')
    top_publishers_per_agency.to_excel(writer, sheet_name='Publishers_by_Agency')